In [1]:

import os
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Unzip the dataset
zip_path = "/content/archive (1).zip"
extract_path = "/content/faces_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted to:", extract_path)
print("Contents:", os.listdir(extract_path))


Dataset extracted to: /content/faces_data
Contents: ['images']


In [2]:

base_dir = os.path.join(extract_path, 'images')
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')

print("Train directory:", train_dir)
print("Validation directory:", val_dir)


Train directory: /content/faces_data/images/train
Validation directory: /content/faces_data/images/validation


In [3]:

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)


Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [4]:

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,626,055 (6.20 MB)

 Trainable params: 1,626,055 (6.20 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:

model.fit(train_generator, validation_data=val_generator, epochs=10)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


901/901 ━━━━━━━━━━━━━━━━━━━━ 189s 208ms/step - accuracy: 0.3222 - loss: 1.7030 - val_accuracy: 0.4519 - val_loss: 1.4242
Epoch 2/10
901/901 ━━━━━━━━━━━━━━━━━━━━ 189s 209ms/step - accuracy: 0.4680 - loss: 1.3868 - val_accuracy: 0.4686 - val_loss: 1.3824
Epoch 3/10
901/901 ━━━━━━━━━━━━━━━━━━━━ 185s 205ms/step - accuracy: 0.5247 - loss: 1.2396 - val_accuracy: 0.5075 - val_loss: 1.2969
Epoch 4/10
901/901 ━━━━━━━━━━━━━━━━━━━━ 185s 206ms/step - accuracy: 0.5770 - loss: 1.1113 - val_accuracy: 0.5167 - val_loss: 1.2941
Epoch 5/10
901/901 ━━━━━━━━━━━━━━━━━━━━ 186s 207ms/step - accuracy: 0.6398 - loss: 0.9726 - val_accuracy: 0.5146 - val_loss: 1.3463
Epoch 6/10
901/901 ━━━━━━━━━━━━━━━━━━━━ 183s 203ms/step - accuracy: 0.7001 - loss: 0.8211 - val_accuracy: 0.5113 - val_loss: 1.4314
Epoch 7/10
901/901 ━━━━━━━━━━━━━━━━━━━━ 185s 205ms/step - accuracy: 0.7533 - loss: 0.6880 - val_accuracy: 0.5190 - val_loss: 1.5185
Epoch 8/10
901/901 ━━━━━━━━━━━━━━━━━━━━ 185s 205ms/step - accuracy: 0.8130 - loss: 0.53

In [6]:
print(train_generator.class_indices)


{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


In [7]:
model.save("emotion_model.h5")
